# POC-Turtle-8: Generalization tests `2016-05-23`
static html: [http://88.99.210.144/data/clustering_2018/html/POC-Turtle-8-Generalization.html](http://88.99.210.144/data/clustering_2018/html/POC-Turtle-8-Generalization.html)  
data: [http://88.99.210.144/data/clustering_2018/Generalization/POC-Turtle/](http://88.99.210.144/data/clustering_2018/Generalization/POC-Turtle/)

## Basic settings

In [1]:
import os, sys, time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import display
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path: sys.path.append(module_path)
src_path = module_path + '/src'
if os.path.exists(src_path) and src_path not in sys.path: 
    sys.path.append(src_path)
lg_path = '/home/oleg/miniconda3/envs/ull4/lib/python3.6/site-packages/linkgrammar'
if os.path.exists(lg_path) and lg_path not in sys.path: 
    sys.path.append(lg_path)
from src.utl.utl import UTC
from src.utl.read_files import check_dir
from src.utl.turtle import html_table
from src.grammar_learner.poc04 import learn_grammar, params, run_learn_grammar
prefix = ''
tmpath = module_path + '/tmp/'
check_dir(tmpath, True, 'none')
print(UTC())

2018-05-23 13:01:55 UTC


## Grammar Learner parameters

In [2]:
kwargs = {
    'left_wall'     :   'LEFT-WALL' ,
    'period'        :   True        ,
    'context'       :   2           ,
    'word_space'    :   'discrete'  ,
    'clustering'    :   'group'     ,
    'grammar_rules' :   2           ,
    'verbose'       :   'min'       ,
    'tmpath'        :   tmpath      , 
    'categories_generalization': 'off',
    'rules_generalization': 'off'   ,
    'test_corpus'   :   module_path + '/data/POC-Turtle/poc-turtle-corpus.txt',
    'reference_path':   module_path + '/data/POC-Turtle/poc-turtle-parses-expected.txt',
    'template_path' :   'poc-turtle',
    'linkage_limit' :   1
}

# Integration test: POC-Turtle 'MST_fixed_manually' 

In [3]:
corpus = 'POC-Turtle'
dataset = 'MST_fixed_manually'
out_dir = module_path + '/output/Generalization-Tests-' + str(UTC())[:10]
kwargs['categories_generalization'] = ''
#kwargs['categories_generalization'] = 'jaccard'
kwargs['rules_generalization'] = ''
kwargs['rules_generalization'] = 'jaccard'
input_parses, output_categories, output_grammar = \
    params(corpus, dataset, module_path, out_dir, **kwargs)

In [4]:
%%capture
r21 = run_learn_grammar(corpus, dataset, module_path, out_dir, **kwargs)

In [5]:
print('\nParse-ability / parse-quality = ', str(r21['parse_ability']) + '% / ' \
    + str(r21['parse_quality'])+'%; ', r21['grammar_rules'], 'Link Grammar rules\n')
#from src.utl.widgets import category_tree
#tree = category_tree(r21['categories_file'], 'max')
def display_tree(response):
    print('Category tree "cat_tree.txt" file:')
    with open(response['tree_file'],'r') as f: x = f.read().splitlines()
    display(html_table([y.split('\t') for y in x]))
display_tree(r21)
#-with open(r21['grammar_file'], 'r') as f: lines = f.read().splitlines()
with open(r21['lg_parse_file'], 'r') as f: lines = f.read().splitlines()
for line in lines[3:-2]: print(line)


Parse-ability / parse-quality =  100% / 100%;  6 Link Grammar rules

Category tree "cat_tree.txt" file:


C01,0,1,0.0,.,0
C02,0,2,0.0,LEFT-WALL,0
C03,0,3,0.0,bird extremity fish,0 0 0
C04,0,9,0.0,eagle herring parrot tuna feather scale fin wing,0 0 0 0 0 0 0 0
,9,4,0.0,eagle herring parrot tuna,0 0 0 0
,9,5,0.0,feather scale,0 0
,9,6,0.0,fin wing,0 0
C05,0,7,0.0,has,0
C06,0,8,0.0,isa,0


tuna isa fish.

    +-C02C04+C04C06+C06C03+C03C01+
    |       |      |      |      |
LEFT-WALL tuna    isa   fish     .

herring isa fish.

    +-C02C04-+C04C06+C06C03+C03C01+
    |        |      |      |      |
LEFT-WALL herring  isa   fish     .

tuna has fin.

    +-C02C04+C04C05+C05C04+C04C01+
    |       |      |      |      |
LEFT-WALL tuna    has    fin     .

herring has fin.

    +-C02C04-+C04C05+C05C04+C04C01+
    |        |      |      |      |
LEFT-WALL herring  has    fin     .

parrot isa bird.

    +-C02C04-+C04C06+C06C03+C03C01+
    |        |      |      |      |
LEFT-WALL parrot   isa   bird     .

eagle isa bird.

    +-C02C04+C04C06+C06C03+C03C01+
    |       |      |      |      |
LEFT-WALL eagle   isa   bird     .

parrot has wing.

    +-C02C04-+C04C05+C05C04+C04C01+
    |        |      |      |      |
LEFT-WALL parrot   has   wing     .

eagle has wing.

    +-C02C04+C04C05+C05C04+C04C01+
    |       |      |      |      |
LEFT-WALL eagle   has   wing     .

fi

# Generalization tests: Disjuncts-ILE-Disjuncts

In [6]:
corpus = 'POC-Turtle'
out_dir = module_path + '/output/Generalization-Tests-' + str(UTC())[:10]
kwargs['verbose'] = 'min'
spaces = 'dILEd'  # Disjuncts-ILE-Disjuncts

## 'MST_fixed_manually' and 'R=6_Weight=6:R' datasets

In [7]:
%%capture
lst = []
for dataset in ['MST_fixed_manually', 'R=6_Weight=6:R']:
    for kwargs['left_wall'] in ['LEFT-WALL', '']:
        if kwargs['left_wall'] == 'LEFT-WALL': 
            lw = 'LW'
        else: lw = ' - '
        for kwargs['period'] in [True, False]:
            if kwargs['period']: 
                dot = ' + '
            else: dot = ' - '
            for g in [('',''), ('jaccard',''), ('','jaccard'), ('jaccard','jaccard')]:
                kwargs['rules_generalization'] = g[0]
                kwargs['categories_generalization'] = g[1]
                if g[0] == '' and g[1] == '': gen = 'none'
                elif g[0] == 'jaccard' and g[1] == '': gen = 'rules'
                elif g[0] == '' and g[1] == 'jaccard': gen = 'categories'
                else: gen = 'both'
                r = run_learn_grammar(corpus, dataset, module_path, out_dir, **kwargs)
                line = [corpus, dataset, lw, dot, gen, spaces, r['grammar_rules'], \
                        str(r['parse_ability'])+'%', str(r['parse_quality'])+'%']
                lst.append(line)

In [8]:
header = ['Corpus','Parsing','LW','"."','Generalization', 'Spaces','Rules','PA','PQ']
display(html_table([header]+lst))

Corpus,Parsing,LW,""".""",Generalization,Spaces,Rules,PA,PQ
POC-Turtle,MST_fixed_manually,LW,+,none,dILEd,8,100%,100%
POC-Turtle,MST_fixed_manually,LW,+,rules,dILEd,6,100%,100%
POC-Turtle,MST_fixed_manually,LW,+,categories,dILEd,6,100%,100%
POC-Turtle,MST_fixed_manually,LW,+,both,dILEd,6,100%,100%
POC-Turtle,MST_fixed_manually,LW,-,none,dILEd,7,100%,100%
POC-Turtle,MST_fixed_manually,LW,-,rules,dILEd,5,100%,100%
POC-Turtle,MST_fixed_manually,LW,-,categories,dILEd,5,100%,100%
POC-Turtle,MST_fixed_manually,LW,-,both,dILEd,5,100%,100%
POC-Turtle,MST_fixed_manually,-,+,none,dILEd,7,100%,100%
POC-Turtle,MST_fixed_manually,-,+,rules,dILEd,5,100%,100%


## 'R=6_Weight=1' and 'LG_ANY_all_parses' datasets

In [9]:
%%capture
lst2 = []
for dataset in ['R=6_Weight=1', 'LG_ANY_all_parses']:
    for kwargs['left_wall'] in ['LEFT-WALL', '']:
        if kwargs['left_wall'] == 'LEFT-WALL': 
            lw = 'LW'
        else: lw = ' - '
        for kwargs['period'] in [True, False]:
            if kwargs['period']: 
                dot = ' + '
            else: dot = ' - '
            for g in [('',''), ('jaccard',''), ('','jaccard'), ('jaccard','jaccard')]:
                kwargs['rules_generalization'] = g[0]
                kwargs['categories_generalization'] = g[1]
                if g[0] == '' and g[1] == '': gen = 'none'
                elif g[0] == 'jaccard' and g[1] == '': gen = 'rules'
                elif g[0] == '' and g[1] == 'jaccard': gen = 'categories'
                else: gen = 'both'
                r = run_learn_grammar(corpus, dataset, module_path, out_dir, **kwargs)
                line = [corpus, dataset, lw, dot, gen, spaces, r['grammar_rules'], \
                        str(r['parse_ability'])+'%', str(r['parse_quality'])+'%']
                lst2.append(line)

In [10]:
display(html_table([header]+lst2))

Corpus,Parsing,LW,""".""",Generalization,Spaces,Rules,PA,PQ
POC-Turtle,R=6_Weight=1,LW,+,none,dILEd,13,67%,33%
POC-Turtle,R=6_Weight=1,LW,+,rules,dILEd,12,67%,33%
POC-Turtle,R=6_Weight=1,LW,+,categories,dILEd,12,67%,33%
POC-Turtle,R=6_Weight=1,LW,+,both,dILEd,12,67%,33%
POC-Turtle,R=6_Weight=1,LW,-,none,dILEd,12,67%,33%
POC-Turtle,R=6_Weight=1,LW,-,rules,dILEd,11,67%,33%
POC-Turtle,R=6_Weight=1,LW,-,categories,dILEd,11,67%,33%
POC-Turtle,R=6_Weight=1,LW,-,both,dILEd,11,67%,33%
POC-Turtle,R=6_Weight=1,-,+,none,dILEd,12,67%,33%
POC-Turtle,R=6_Weight=1,-,+,rules,dILEd,10,83%,42%


# Generalization tests: "Connectors-DRK-Disjuncts"

## Test all parses, with/without LEFT-WALL, period, rules generalization

In [11]:
%%capture
spaces = 'cDRKd'  # "Connectors-DRK-Disjuncts"
kwargs['context'] = 1
kwargs['word_space'] = 'vectors'
kwargs['clustering'] = 'kmeans'
kwargs['categories_generalization'] = 'off'
kwargs['grammar_rules'] = 2 
lst3 = []
for dataset in ['MST_fixed_manually', 'R=6_Weight=6:R', 'R=6_Weight=1', 'LG_ANY_all_parses']:
    for kwargs['left_wall'] in ['LEFT-WALL', '']:
        if kwargs['left_wall'] == 'LEFT-WALL': 
            lw = 'LW'
        else: lw = ' - '
        for kwargs['period'] in [True, False]:
            if kwargs['period']: 
                dot = ' + '
            else: dot = ' - '
            for g in [('',''), ('jaccard', '')]:
                kwargs['rules_generalization'] = g[0]
                kwargs['categories_generalization'] = g[1]
                if g[0] == '' and g[1] == '': gen = 'none'
                elif g[0] == 'jaccard' and g[1] == '': gen = 'rules'
                elif g[0] == '' and g[1] == 'jaccard': gen = 'categories'
                else: gen = 'both'
                r = run_learn_grammar(corpus, dataset, module_path, out_dir, **kwargs)
                line = [corpus, dataset, lw, dot, gen, spaces, r['grammar_rules'], \
                        str(r['parse_ability'])+'%', str(r['parse_quality'])+'%']
                lst3.append(line)

In [12]:
display(html_table([header]+lst3))

Corpus,Parsing,LW,""".""",Generalization,Spaces,Rules,PA,PQ
POC-Turtle,MST_fixed_manually,LW,+,none,cDRKd,4,100%,100%
POC-Turtle,MST_fixed_manually,LW,+,rules,cDRKd,2,100%,100%
POC-Turtle,MST_fixed_manually,LW,-,none,cDRKd,4,100%,100%
POC-Turtle,MST_fixed_manually,LW,-,rules,cDRKd,2,100%,100%
POC-Turtle,MST_fixed_manually,-,+,none,cDRKd,4,100%,100%
POC-Turtle,MST_fixed_manually,-,+,rules,cDRKd,2,100%,100%
POC-Turtle,MST_fixed_manually,-,-,none,cDRKd,4,100%,100%
POC-Turtle,MST_fixed_manually,-,-,rules,cDRKd,2,100%,100%
POC-Turtle,R=6_Weight=6:R,LW,+,none,cDRKd,3,100%,92%
POC-Turtle,R=6_Weight=6:R,LW,+,rules,cDRKd,2,100%,100%


## MST_fixed manually, no LEFT-WALL, no '.' -- 2 rules!
**2 word categories: verbs and nouns, 2 rules form the grammar with 100% parse-ability and quality**

In [13]:
%%capture
kwargs['rules_generalization'] = 'jaccard'
kwargs['left_wall'] = ''
kwargs['period'] = False
dataset = 'MST_fixed_manually'
r32 = run_learn_grammar(corpus, dataset, module_path, out_dir, **kwargs)

In [14]:
print('Parse-ability / parse-quality = ', str(r32['parse_ability']) + '% / ' \
    + str(r32['parse_quality'])+'%; ', r32['grammar_rules'], 'Link Grammar rules\n')
#tree = category_tree(r32['categories_file'], 'max')
with open(r32['lg_parse_file'], 'r') as f: lines = f.read().splitlines()
for line in lines[3:-2]: print(line)

Parse-ability / parse-quality =  100% / 100%;  2 Link Grammar rules

tuna isa fish.

  +C02C01+C01C02+
  |      |      |
tuna    isa   fish [.]

herring isa fish.

   +C02C01+C01C02+
   |      |      |
herring  isa   fish [.]

tuna has fin.

  +C02C01+C01C02+
  |      |      |
tuna    has    fin [.]

herring has fin.

   +C02C01+C01C02+
   |      |      |
herring  has    fin [.]

parrot isa bird.

   +C02C01+C01C02+
   |      |      |
parrot   isa   bird [.]

eagle isa bird.

  +C02C01+C01C02+
  |      |      |
eagle   isa   bird [.]

parrot has wing.

   +C02C01+C01C02+
   |      |      |
parrot   has   wing [.]

eagle has wing.

  +C02C01+C01C02+
  |      |      |
eagle   has   wing [.]

fin isa extremity.

 +C02C01+C01C02+
 |      |      |
fin    isa extremity [.]

wing isa extremity.

  +C02C01+C01C02+
  |      |      |
wing    isa extremity [.]

fin has scale.

 +C02C01+C01C02+
 |      |      |
fin    has   scale [.]

wing has feather.

  +C02C01+C01C02+
  |      |      |
wing    

## 'R=6_Weight=6:R', 2 generalized rules: 
**LEFT-WALL and period added to the verbs cluster -- over-generalization?**

In [15]:
%%capture
dataset = 'R=6_Weight=6:R'
kwargs['rules_generalization'] = 'jaccard'
kwargs['left_wall'] = 'LEFT-WALL'
kwargs['period'] = True
r33 = run_learn_grammar(corpus, dataset, module_path, out_dir, **kwargs)

In [16]:
print('Parse-ability / parse-quality = ', str(r33['parse_ability']) + '% / ' \
    + str(r33['parse_quality'])+'%; ', r33['grammar_rules'], 'Link Grammar rules\n')
#tree = category_tree(r32['categories_file'], 'max')
with open(r33['lg_parse_file'], 'r') as f: lines = f.read().splitlines()
for line in lines[3:-2]: print(line)

Parse-ability / parse-quality =  100% / 100%;  2 Link Grammar rules

tuna isa fish.

    +-C02C01+C01C02+C02C01+C01C02+
    |       |      |      |      |
LEFT-WALL tuna    isa   fish     .

herring isa fish.

    +-C02C01-+C01C02+C02C01+C01C02+
    |        |      |      |      |
LEFT-WALL herring  isa   fish     .

tuna has fin.

    +-C02C01+C01C02+C02C02+C02C02+
    |       |      |      |      |
LEFT-WALL tuna    has    fin     .

herring has fin.

    +-C02C01-+C01C02+C02C02+C02C02+
    |        |      |      |      |
LEFT-WALL herring  has    fin     .

parrot isa bird.

    +-C02C01-+C01C02+C02C01+C01C02+
    |        |      |      |      |
LEFT-WALL parrot   isa   bird     .

eagle isa bird.

    +-C02C01+C01C02+C02C01+C01C02+
    |       |      |      |      |
LEFT-WALL eagle   isa   bird     .

parrot has wing.

    +-C02C01-+C01C02+C02C02+C02C02+
    |        |      |      |      |
LEFT-WALL parrot   has   wing     .

eagle has wing.

    +-C02C01+C01C02+C02C02+C02C02+
    